<a href="https://colab.research.google.com/github/manavraj01/Rossmann_Store_Sales/blob/first_upload/Rossmann_Store_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy pandas scikit-learn seaborn plotly opendatasets --quiet

In [2]:
import opendatasets as od

In [3]:
dataset_url = "https://www.kaggle.com/competitions/rossmann-store-sales/data";
od.download(dataset_url)

100%|██████████| 6.99M/6.99M [00:00<00:00, 809MB/s]


Extracting archive ./rossmann-store-sales/rossmann-store-sales.zip to ./rossmann-store-sales


In [24]:
import os
dataset = os.listdir('rossmann-store-sales')
test_dataset = dataset[0]
dataset

['train.csv', 'test.csv', 'sample_submission.csv', 'store.csv']

In [7]:
import pandas as pd

In [10]:
test_df = pd.read_csv(f'rossmann-store-sales/{test_dataset}',low_memory= False)

In [11]:
test_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [12]:
import datetime

def findWeekend(date,format = "%Y-%m-%d"):
  date_obj = datetime.datetime.strptime(date,format)

  if date_obj.weekday()>=5:
    return True
  return  False

In [16]:
#Add another column in test dataframe
test_df["Weekend"] = test_df["Date"].apply(findWeekend)
test_df.drop(columns = ["Date"],inplace=True)
test_df.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Weekend
0,1,5,5263,555,1,1,0,1,False
1,2,5,6064,625,1,1,0,1,False
2,3,5,8314,821,1,1,0,1,False
3,4,5,13995,1498,1,1,0,1,False
4,5,5,4822,559,1,1,0,1,False


In [20]:
test_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Store,1017209.0,558.429727,321.908651,1.0,280.0,558.0,838.0,1115.0
DayOfWeek,1017209.0,3.998341,1.997391,1.0,2.0,4.0,6.0,7.0
Sales,1017209.0,5773.818972,3849.926175,0.0,3727.0,5744.0,7856.0,41551.0
Customers,1017209.0,633.145946,464.411734,0.0,405.0,609.0,837.0,7388.0
Open,1017209.0,0.830107,0.375539,0.0,1.0,1.0,1.0,1.0
Promo,1017209.0,0.381515,0.485759,0.0,0.0,0.0,1.0,1.0
SchoolHoliday,1017209.0,0.178647,0.383056,0.0,0.0,0.0,0.0,1.0


In [22]:
#Remove data when store was closed
test_df = test_df[test_df["Open"]==1]
test_df.drop(columns="Open",inplace=True)
test_df.head()

,Store,DayOfWeek,Sales,Customers,Promo,StateHoliday,SchoolHoliday,Weekend
0,1,5,5263,555,1,0,1,False
1,2,5,6064,625,1,0,1,False
2,3,5,8314,821,1,0,1,False
3,4,5,13995,1498,1,0,1,False
4,5,5,4822,559,1,0,1,False


In [26]:
#Let us fetch data from store
store_dataset = dataset[3]
store_df = pd.read_csv(f'rossmann-store-sales/{store_dataset}',low_memory=False)
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [27]:
#Let us connect test_df with store df
test_df = test_df.merge(store_df,on="Store",how="left")
test_df

,Store,DayOfWeek,Sales,Customers,Promo,StateHoliday,SchoolHoliday,Weekend,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,5263,555,1,0,1,False,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,6064,625,1,0,1,False,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,8314,821,1,0,1,False,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,13995,1498,1,0,1,False,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,4822,559,1,0,1,False,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844387,682,2,3375,566,0,a,1,False,b,a,150.0,9.0,2006.0,0,NaN,NaN,NaN
844388,733,2,10765,2377,0,a,1,False,b,b,860.0,10.0,1999.0,0,NaN,NaN,NaN
844389,769,2,5035,1248,0,a,1,False,b,b,840.0,NaN,NaN,1,48.0,2012.0,"Jan,Apr,Jul,Oct"
844390,948,2,4491,1039,0,a,1,False,b,b,1430.0,NaN,NaN,0,NaN,NaN,NaN


In [28]:
#Let us check if any duplicate rows are present
test_df.duplicated().sum()

np.int64(159)

In [30]:
test_df.drop_duplicates()

,Store,DayOfWeek,Sales,Customers,Promo,StateHoliday,SchoolHoliday,Weekend,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,5263,555,1,0,1,False,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,6064,625,1,0,1,False,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,8314,821,1,0,1,False,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,13995,1498,1,0,1,False,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,4822,559,1,0,1,False,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844387,682,2,3375,566,0,a,1,False,b,a,150.0,9.0,2006.0,0,NaN,NaN,NaN
844388,733,2,10765,2377,0,a,1,False,b,b,860.0,10.0,1999.0,0,NaN,NaN,NaN
844389,769,2,5035,1248,0,a,1,False,b,b,840.0,NaN,NaN,1,48.0,2012.0,"Jan,Apr,Jul,Oct"
844390,948,2,4491,1039,0,a,1,False,b,b,1430.0,NaN,NaN,0,NaN,NaN,NaN


In [32]:
test_df.shape

(844392, 17)

In [34]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 844392 entries, 0 to 844391
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Store                      844392 non-null  int64  
 1   DayOfWeek                  844392 non-null  int64  
 2   Sales                      844392 non-null  int64  
 3   Customers                  844392 non-null  int64  
 4   Promo                      844392 non-null  int64  
 5   StateHoliday               844392 non-null  object 
 6   SchoolHoliday              844392 non-null  int64  
 7   Weekend                    844392 non-null  bool   
 8   StoreType                  844392 non-null  object 
 9   Assortment                 844392 non-null  object 
 10  CompetitionDistance        842206 non-null  float64
 11  CompetitionOpenSinceMonth  575773 non-null  float64
 12  CompetitionOpenSinceYear   575773 non-null  float64
 13  Promo2                     84

In [35]:
#Let us find numerical and object or boolean cols
numerical_cols = test_df.select_dtypes(include=["int64","float64"]).columns.tolist()
categorical_cols = test_df.select_dtypes(include=["object","bool"]).columns.tolist()

In [57]:
input_cols = numerical_cols + categorical_cols
target_cols = "Sales"
#input_cols.remove(target_cols)


In [58]:
#Data visualisation

#Let us find it first using correlation

corr = test_df.corr(numeric_only=True)[target_cols].sort_values(ascending=False)
corr

,Sales
Sales,1.000000
Customers,0.823597
Promo,0.368145
Promo2,-0.127596
Weekend,-0.151780
DayOfWeek,-0.178736


In [59]:
strong_feats = corr[abs(corr)>.1].index.tolist()
strong_feats

['Sales', 'Customers', 'Promo', 'Promo2', 'Weekend', 'DayOfWeek']

In [60]:
main_df = test_df[strong_feats + [target_cols]]

In [61]:
#Let us now do the imputer and scaling thing
numeric_feats = [col for col in strong_feats if col in numerical_cols]
categorical_feats = [col for col in strong_feats if col in categorical_cols]

In [62]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="mean").fit(main_df[numeric_feats])
main_df[numeric_feats]= imputer.transform(main_df[numeric_feats])

/tmp/ipython-input-609279878.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df[numeric_feats]= imputer.transform(main_df[numeric_feats])


In [63]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(main_df[numeric_feats])
main_df[numeric_feats] = scaler.transform(main_df[numeric_feats])

/tmp/ipython-input-970391769.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df[numeric_feats] = scaler.transform(main_df[numeric_feats])


In [64]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore').fit(main_df[categorical_feats])
encoded_cols = list(encoder.get_feature_names_out(categorical_feats))
main_df[encoded_cols] = encoder.transform(main_df[categorical_feats]).toarray()

NotImplementedError: 

In [67]:
X = main_df.drop(columns=[target_cols])

# Target (the value to predict)
y = main_df[target_cols]

In [69]:
from sklearn.model_selection import train_test_split

# 70% train, 30% temp (to be split further into val + test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42
)


In [70]:
# Split the 30% equally → 15% val, 15% test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)


In [71]:
print("Train:", X_train.shape, y_train.shape)
print("Validation:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)


Train: (591074, 5) (591074, 2)
Validation: (126659, 5) (126659, 2)
Test: (126659, 5) (126659, 2)


In [72]:
import numpy as np

def rmspe(y_true, y_pred):
    # avoid division by zero
    mask = y_true != 0
    return np.sqrt(np.mean( ((y_true[mask] - y_pred[mask]) / y_true[mask]) ** 2 ))


In [73]:
from sklearn.linear_model import LinearRegression

# Initialize and fit model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predict
y_val_pred = lr.predict(X_val)
y_test_pred = lr.predict(X_test)


In [74]:
print("Validation RMSPE:", rmspe(y_val.values, y_val_pred))
print("Test RMSPE:", rmspe(y_test.values, y_test_pred))


Validation RMSPE: 0.27162565040004033
Test RMSPE: 0.251648388268703


In [76]:
from sklearn.ensemble import RandomForestRegressor

# Initialize model
rf = RandomForestRegressor(
    n_estimators=200,    # number of trees
    max_depth=None,      # let trees expand fully
    random_state=42,
    n_jobs=-1            # use all CPU cores
)

# Fit
rf.fit(X_train, y_train)

# Predict
y_val_pred_rf = rf.predict(X_val)
y_test_pred_rf = rf.predict(X_test)


In [77]:
print("Validation RMSPE (RF):", rmspe(y_val.values, y_val_pred_rf))
print("Test RMSPE (RF):", rmspe(y_test.values, y_test_pred_rf))

print("Validation R² (RF):", rf.score(X_val, y_val))
print("Test R² (RF):", rf.score(X_test, y_test))


Validation RMSPE (RF): 0.21792496404607725
Test RMSPE (RF): 0.21953282341692526
Validation R² (RF): 0.7541312411596659
Test R² (RF): 0.7476096968475232


In [79]:
!pip install xgboost


In [83]:
from xgboost import XGBRegressor

# Initialize model
xgb = XGBRegressor(
    n_estimators=500,       # number of boosting rounds (trees)
    learning_rate=0.05,     # step size shrinkage
    max_depth=8,            # tree depth (controls complexity)
    subsample=0.8,          # row sampling
    colsample_bytree=0.8,   # feature sampling
    random_state=42,
    n_jobs=-1
)

xgb.fit(
    X_train.values, y_train.values,
    eval_set=[(X_val.values, y_val.values)],
    verbose=False
)




XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=-1, num_parallel_tree=None, ...)

In [84]:
y_val_pred_xgb = xgb.predict(X_val)
y_test_pred_xgb = xgb.predict(X_test)

print("Validation RMSPE (XGB):", rmspe(y_val.values, y_val_pred_xgb))
print("Test RMSPE (XGB):", rmspe(y_test.values, y_test_pred_xgb))

print("Validation R² (XGB):", xgb.score(X_val, y_val))
print("Test R² (XGB):", xgb.score(X_test, y_test))


Validation RMSPE (XGB): 0.2315068860365718
Test RMSPE (XGB): 0.21407035310148215
Validation R² (XGB): 0.776066780090332
Test R² (XGB): 0.770743727684021
